In [ ]:
!pip install yfinance


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
# Télécharger les données
data = yf.download("AAPL", start="2024-01-01", end="2024-07-01")

# Vérifie les colonnes disponibles
print("Colonnes:", data.columns)


In [ ]:
# Aplatir les colonnes si MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

# Calcul des rendements
data['Return'] = data['Close'].pct_change()

# Variables pour régression
data['Return_t-1'] = data['Return'].shift(1)
data.dropna(inplace=True)

# Variables explicatives et cible
x = data['Return_t-1'].values
y = data['Return'].values

In [ ]:
def linear_regression_gd(x, y, learning_rate=0.1, iterations=1000):
    slope = intercept = 0
    n = len(x)
    
    for i in range(iterations):
        y_pred = slope * x + intercept
        cost = (1/n) * sum((y - y_pred)**2)
        
        slope_grad = -(2/n) * sum(x * (y - y_pred))
        intercept_grad = -(2/n) * sum(y - y_pred)
        
        slope = slope - learning_rate * slope_grad
        intercept = intercept - learning_rate * intercept_grad
        
        if i % 100 == 0:
            print(f"Iteration {i}: slope={slope:.4f}, intercept={intercept:.4f}, cost={cost:.6f}")
    
    return slope, intercept

In [ ]:
slope, intercept = linear_regression_gd(x, y)

# Affichage des résultats
plt.scatter(x, y, color='blue', label='Data')
plt.plot(x, slope * x + intercept, color='red', label='Prediction')
plt.xlabel("Rendement t-1")
plt.ylabel("Rendement t")
plt.title("Régression linéaire sur les rendements journaliers")
plt.legend()
plt.show()


In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression

# Reshaper x (important pour sklearn)
x = x.reshape(-1, 1)

# Créer et entraîner le modèle
model = LinearRegression()
model.fit(x, y)

# Coefficients
slope = model.coef_[0]
intercept = model.intercept_

print(f"Pente (coefficient): {slope}")
print(f"Ordonnée à l'origine (intercept): {intercept}")

In [ ]:
# Prédictions
y_pred = model.predict(x)

# Tracer
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', label='Data')
plt.plot(x, y_pred, color='red', label='Régression')
plt.xlabel("Rendement t-1")
plt.ylabel("Rendement t")
plt.title("Régression linéaire simple sur les rendements journaliers")
plt.legend()
plt.grid(True)
plt.show()